In [1]:
import torch
import numpy as np
from pathlib import Path
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler, LabelEncoder
import logging
from datetime import datetime

from pretrain import pretrain
from finetune import finetune, FineTuningConfig
from config import *
from data_loader import load_and_preprocess_har_data
import os
import gdown

In [2]:
def download_file_from_google_drive(file_id, destination):
    url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(url, destination, quiet=False)

if __name__ == "__main__":
    # Create data directory if it doesn't exist
    if not os.path.exists('data'):
        os.makedirs('data')

    # Define file IDs and destination paths
    files_to_download = [
        {
            'file_id': '1vK1flNSOMADCmYnxQjSHoq3FupZ42Anq',
            'dest_path': 'data/X_unlabeled.npy'
        },
        {
            'file_id': '1sXJX-1Tge5voIPXxFhMene1HLKhfp9gm',
            'dest_path': 'data/X.npy'
        },
        {
            'file_id': '1jgNQ2M5mrtjqo7V_fCpHbGigslc1jjOC',
            'dest_path': 'data/y.npy'
        },
        {
            'file_id': '1ZXSvlVzxipCP_EDsUMtPE4TM0ygVMxaJ',
            'dest_path': 'data/sub.npy'
        },
    ]

    # Download each file
    for file_info in files_to_download:
        print(f"Downloading {file_info['dest_path']}...")
        download_file_from_google_drive(file_info['file_id'], file_info['dest_path'])
        print(f"Downloaded {file_info['dest_path']}.\n")


Downloading...
From (original): https://drive.google.com/uc?id=1vK1flNSOMADCmYnxQjSHoq3FupZ42Anq
From (redirected): https://drive.google.com/uc?id=1vK1flNSOMADCmYnxQjSHoq3FupZ42Anq&confirm=t&uuid=817e0396-b7c7-470b-93d9-fbaf3899cc56
To: /mnt/batch/tasks/shared/LS_root/mounts/clusters/habibirani-gpu/code/Users/habibirani/t-nnclr/data/X_unlabeled.npy
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

Downloaded data/X_unlabeled.npy.



Downloading...
From: https://drive.google.com/uc?id=1sXJX-1Tge5voIPXxFhMene1HLKhfp9gm
To: /mnt/batch/tasks/shared/LS_root/mounts/clusters/habibirani-gpu/code/Users/habibirani/t-nnclr/data/X.npy
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

Downloaded data/X.npy.



Downloading...
From: https://drive.google.com/uc?id=1jgNQ2M5mrtjqo7V_fCpHbGigslc1jjOC
To: /mnt/batch/tasks/shared/LS_root/mounts/clusters/habibirani-gpu/code/Users/habibirani/t-nnclr/data/y.npy
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

Downloaded data/y.npy.



Downloading...
From: https://drive.google.com/uc?id=1ZXSvlVzxipCP_EDsUMtPE4TM0ygVMxaJ
To: /mnt/batch/tasks/shared/LS_root/mounts/clusters/habibirani-gpu/code/Users/habibirani/t-nnclr/data/sub.npy
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

Downloaded data/sub.npy.



In [3]:
def load_and_preprocess_har_data(data_dir='./data', logger=None):
    """
    Load and preprocess HAR dataset.
    
    Args:
        data_dir: Directory containing the data files
        logger: Optional logger instance
    
    Returns:
        tuple: (X_unlabeled, X_labeled, y, subjects)
            - X_unlabeled: Unlabeled data array
            - X_labeled: Labeled data array
            - y: Labels array
            - subjects: Subject IDs array
    """
    try:
        data_dir = Path(data_dir)
        
        # Load data
        X_unlabeled = np.load(data_dir / 'X_unlabeled.npy')
        # X_unlabeled = np.load(data_dir / 'X.npy')
        X_labeled = np.load(data_dir / 'X.npy')
        y = np.load(data_dir / 'y.npy').squeeze()
        subjects = np.load(data_dir / 'sub.npy').squeeze()
        
        # Use only first 4 channels of the data
        X_unlabeled = X_unlabeled[:, :, :4]
        X_labeled = X_labeled[:, :, :4]
        
        if logger:
            logger.info(f"Loaded data shapes:")
            logger.info(f"X_unlabeled: {X_unlabeled.shape}")
            logger.info(f"X_labeled: {X_labeled.shape}")
            logger.info(f"y: {y.shape}")
            logger.info(f"subjects: {subjects.shape}")
            logger.info(f"Number of unique subjects: {len(np.unique(subjects))}")
            logger.info(f"Number of unique classes: {len(np.unique(y))}")
        
        # Convert labels to integers
        le = LabelEncoder()
        y = le.fit_transform(y)
        if logger:
            logger.info("Labels encoded successfully")
        
        # Standardize the data
        scaler = StandardScaler()
        
        # Reshape for standardization
        orig_shape_unlabeled = X_unlabeled.shape
        orig_shape_labeled = X_labeled.shape
        
        # Combine all data for computing statistics
        combined = np.vstack([
            X_unlabeled.reshape(-1, X_unlabeled.shape[-1]),
            X_labeled.reshape(-1, X_labeled.shape[-1])
        ])
        
        # Fit on combined data
        scaler.fit(combined)
        
        # Transform separately and reshape back
        X_unlabeled = scaler.transform(
            X_unlabeled.reshape(-1, X_unlabeled.shape[-1])
        ).reshape(orig_shape_unlabeled)
        
        X_labeled = scaler.transform(
            X_labeled.reshape(-1, X_labeled.shape[-1])
        ).reshape(orig_shape_labeled)
        
        if logger:
            logger.info("Data standardization completed")
        
        return X_unlabeled, X_labeled, y, subjects
        
    except Exception as e:
        if logger:
            logger.error(f"Error loading data: {str(e)}")
        raise

if __name__ == "__main__":
    # Test data loading
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    
    data_dir = Path.cwd() / 'data'
    X_unlabeled, X_labeled, y, subjects = load_and_preprocess_har_data(data_dir, logger)
    
    print("\nData loading test completed successfully!")
    print(f"Unlabeled data shape: {X_unlabeled.shape}")
    print(f"Labeled data shape: {X_labeled.shape}")
    print(f"Labels shape: {y.shape}")
    print(f"Subjects shape: {subjects.shape}")

INFO:__main__:Loaded data shapes:
INFO:__main__:X_unlabeled: (331964, 96, 4)
INFO:__main__:X_labeled: (3168, 96, 4)
INFO:__main__:y: (3168,)
INFO:__main__:subjects: (3168,)
INFO:__main__:Number of unique subjects: 3
INFO:__main__:Number of unique classes: 6
INFO:__main__:Labels encoded successfully
INFO:__main__:Data standardization completed



Data loading test completed successfully!
Unlabeled data shape: (331964, 96, 4)
Labeled data shape: (3168, 96, 4)
Labels shape: (3168,)
Subjects shape: (3168,)


In [4]:
def setup_logging(experiment_dir):
    """Setup logging configuration."""
    experiment_dir.mkdir(parents=True, exist_ok=True)
    
    log_file = experiment_dir / 'experiment.log'
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger(__name__)

def run_cross_validation_experiment(
    data_dir='./data',
    experiment_dir=None,
    pretrain_epochs=PRETRAIN_EPOCHS,
    finetune_epochs=FINETUNE_EPOCHS,
    freeze_encoder=False,
    encoder_lr=LEARNING_RATE,
    head_lr=LEARNING_RATE
):
    """
    Run NNCLR experiment with Leave-One-Subject-Out cross validation.
    
    Args:
        data_dir: Directory containing the data files
        experiment_dir: Directory to save experiment results
        pretrain_epochs: Number of pretraining epochs
        finetune_epochs: Number of finetuning epochs
        freeze_encoder: Whether to freeze encoder during fine-tuning
        encoder_lr: Learning rate for encoder during fine-tuning
        head_lr: Learning rate for classification head during fine-tuning
    
    Returns:
        dict: Results dictionary containing accuracies and confusion matrices
    """
    # Setup experiment directory and logging
    if experiment_dir is None:
        experiment_dir = Path.cwd() / 'experiments' / datetime.now().strftime('%Y%m%d_%H%M%S')
    else:
        experiment_dir = Path(experiment_dir)
    
    logger = setup_logging(experiment_dir)
    
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logger.info(f"Using device: {device}")
    
    # Load data
    X_unlabeled, X_labeled, y, subjects = load_and_preprocess_har_data(data_dir, logger)
    
    # Create fine-tuning configuration
    finetune_config = FineTuningConfig(
        freeze_encoder=freeze_encoder,
        encoder_lr=encoder_lr,
        head_lr=head_lr,
        num_epochs=finetune_epochs
    )
    
    # Perform pretraining once using only unlabeled data
    logger.info("Starting pretraining phase...")
    pretrain_save_path = experiment_dir / 'pretrained_model.pt'
    
    pretrained_model, pretrain_history = pretrain(
        X_unlabeled=X_unlabeled,
        num_epochs=pretrain_epochs,
        device=device,
        save_path=pretrain_save_path
    )
    
    # Save pretraining history
    np.save(experiment_dir / 'pretrain_history.npy', pretrain_history)
    
    # Setup cross-validation
    cv = LeaveOneGroupOut()
    splits = list(cv.split(X_labeled, y, subjects))
    logger.info(f"Running Leave-One-Subject-Out CV with {len(splits)} folds")
    
    # Rest of the code remains unchanged
    results = {
        'fold_accuracies': [],
        'confusion_matrices': [],
        'finetune_histories': []
    }
    
    # Run cross-validation
    for fold, (train_idx, test_idx) in enumerate(splits):
        fold_dir = experiment_dir / f'fold_{fold}'
        fold_dir.mkdir(exist_ok=True)
        
        logger.info(f"\nStarting Fold {fold + 1}/{len(splits)}")
        
        # Split data for this fold
        X_train = X_labeled[train_idx]
        y_train = y[train_idx]
        X_test = X_labeled[test_idx]
        y_test = y[test_idx]
        
        # Finetuning phase
        logger.info("Starting finetuning phase...")
        finetune_save_path = fold_dir / 'finetuned_model.pt'
        
        # Fine-tune using pretrained weights
        model, finetune_history = finetune(
            pretrained_model_path=pretrain_save_path,
            X_train=X_train,
            y_train=y_train,
            X_test=X_test,
            y_test=y_test,
            config=finetune_config,
            device=device,
            save_path=finetune_save_path
        )
        
        # Store results for this fold
        results['fold_accuracies'].append(finetune_history['best_acc'])
        results['confusion_matrices'].append(finetune_history['confusion_matrix'])
        results['finetune_histories'].append(finetune_history)
        
        logger.info(f"Fold {fold + 1} - Best Test Accuracy: {finetune_history['best_acc']:.2f}%")
    
    # Calculate and log final results
    accuracies = np.array(results['fold_accuracies'])
    cumulative_confusion = sum(results['confusion_matrices'])
    
    logger.info("\nFinal Results:")
    logger.info(f"Mean Accuracy: {accuracies.mean():.2f}% +/- {accuracies.std():.2f}%")
    logger.info("\nCumulative Confusion Matrix:")
    logger.info("\n" + str(cumulative_confusion))
    
    # Save final results
    np.save(experiment_dir / 'accuracies.npy', accuracies)
    np.save(experiment_dir / 'confusion_matrices.npy', results['confusion_matrices'])
    np.save(experiment_dir / 'cumulative_confusion.npy', cumulative_confusion)
    
    # Save summary statistics
    with open(experiment_dir / 'summary.txt', 'w') as f:
        f.write("Experiment Configuration:\n")
        f.write(f"Freeze encoder: {freeze_encoder}\n")
        f.write(f"Encoder learning rate: {encoder_lr}\n")
        f.write(f"Head learning rate: {head_lr}\n")
        f.write(f"Pretrain epochs: {pretrain_epochs}\n")
        f.write(f"Finetune epochs: {finetune_epochs}\n\n")
        f.write(f"Mean Accuracy: {accuracies.mean():.2f}% +/- {accuracies.std():.2f}%\n")
        f.write(f"Individual Fold Accuracies: {accuracies.tolist()}\n")
        f.write(f"\nCumulative Confusion Matrix:\n{cumulative_confusion}")
    
    return results


In [5]:
if __name__ == "__main__":
    # Set paths
    data_dir = Path.cwd() / 'data'
    experiment_dir = Path.cwd() / 'experiments' / datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Run experiment with specific configuration
    results = run_cross_validation_experiment(
        data_dir=data_dir,
        experiment_dir=experiment_dir,
        pretrain_epochs=PRETRAIN_EPOCHS,
        finetune_epochs=FINETUNE_EPOCHS,
        freeze_encoder=False,
        encoder_lr=LEARNING_RATE,
        head_lr=LEARNING_RATE
    )
    
    print("\nExperiment completed!")
    print(f"Results saved to {experiment_dir}")

INFO:__main__:Using device: cuda


INFO:__main__:Loaded data shapes:
INFO:__main__:X_unlabeled: (331964, 96, 4)
INFO:__main__:X_labeled: (3168, 96, 4)
INFO:__main__:y: (3168,)
INFO:__main__:subjects: (3168,)
INFO:__main__:Number of unique subjects: 3
INFO:__main__:Number of unique classes: 6
INFO:__main__:Labels encoded successfully
INFO:__main__:Data standardization completed
INFO:__main__:Starting pretraining phase...


Pretraining on device: cuda
Unlabeled data shape: (331964, 96, 4)

Starting pretraining...


Epoch 1/2: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5186/5186 [01:56<00:00, 44.45it/s, loss=0.5881]



Epoch 1/2
Loss: 0.5881


...


Epoch 2/2
Train Loss: 0.5954 | Train Acc: 77.55%
Test Loss: 0.8624 | Test Acc: 56.89%
Best Test Acc: 57.09%
Finetuning on device: cuda
Training data shape: (2077, 96, 4)
Test data shape: (1091, 96, 4)
Encoder frozen: False
Encoder LR: 0.001
Head LR: 0.001

Starting finetuning...


Epoch 1/2: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 87.97it/s, loss=1.2216, acc=61.39%]



Epoch 1/2
Train Loss: 1.0365 | Train Acc: 61.39%
Test Loss: 0.8326 | Test Acc: 67.00%
Best Test Acc: 67.00%


Epoch 2/2: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 88.88it/s, loss=0.7015, acc=74.92%]



Epoch 2/2
Train Loss: 0.5952 | Train Acc: 74.92%
Test Loss: 0.7641 | Test Acc: 70.12%
Best Test Acc: 70.12%

Experiment completed!
Results saved to /mnt/batch/tasks/shared/LS_root/mounts/clusters/habibirani-gpu/code/Users/habibirani/t-nnclr/experiments/20250312_165207
